In [1]:
import warnings
warnings.filterwarnings('ignore')

import holoviews as hv
hv.notebook_extension('bokeh','matplotlib')

import param

import numpy as np
import pandas as pd
print("HV version", hv.__version__ )

HV version 1.8.0


# Bokeh App 2: Euler's Method

Objective: create a template for a display with information flowing from server to client and back

We will consider Euler's method for $y' = f( x, y )$

Using the approximation $y' \approx \frac{y(x+h)-y(x)}{h}$ for some small value of $h$ and evaluation on a set of x grid points $x_n = x_0 + n h$<br>
we obtain<br><br>
$y_{n+1} = y_n + h f( x_n, y_n)$, an iterative scheme with starting point $(x_0, y_0)$ 

## Develop the display, avoiding cell magic options and providing a means of sizing the display 

### The background

In [9]:
def background( func=lambda x,y: np.sin(x*y), size=(500,500)):
    ''' Given the ODE y'=f(x,y)
             bg,vec,xaxis_line,yaxis_line = background()
        returns a grayscale image of the slopes y',
                a vector field representation of the slopes, and
                a set of axis lines for -5<x<5, -5<y<5
    '''
    
    # compute the data
    vals = np.linspace( -5, 5, num=150 )
    X, Y = np.meshgrid(vals, vals)

    clines  = func(X,Y)                    # f(x,y)
    theta   = np.arctan2( clines, 1)       # angle of the slope y' at x,y

    # Obtain the vector field (subsample the grid)
    h,w=size
    vf_opts = dict(size_index=3, height=h, width=w, xticks=9, yticks=9)
    vec_field = hv.VectorField((vals[::3],vals[::3],theta[::3,::3],0*clines[::3,::3]+1) ).opts(plot=vf_opts)
    
    # overlay the vector field with a gray scale image:
    #     this is tricky: we need RGB to be able to set alpha
    #        and the data must be normalized to [0,1]
    #   to work correctly the RGB constructor RGB((x,y, r,g,b,a), vdims=[]) must set
    #        the vdim names 'R','G','B','A'
    def norm(arr):
        arr = (arr-arr.min())
        return arr/arr.max()

    normXY    = norm(clines)
    img_field = hv.RGB( (vals, vals, normXY, normXY, normXY, 0*clines+0.3), vdims=['R','G','B','A'] )\
                .opts(style=dict(width=600, height=600, cmap='gray'), plot=dict(shared_axes=False))

    # finally, we add the axes as VLine, HLine and return an array of the plot Elements
    hv_opts = dict( color='green', alpha=0.5)
    return [img_field,vec_field, hv.HLine(0).opts(style=hv_opts),
                hv.VLine(0).opts(style=hv_opts)]

#Test it:
hv.Overlay(background(size=(400,400))).relabel("y' = sin(x y)" ) +\
hv.Overlay(background(func=lambda x,y: x*np.sin(5*y), size=(400,400))).relabel("y' = x sin(5 y)" )

:Layout
   .Overlay.Y_apostrophe_equals_sin_left_parenthesis_x_y_right_parenthesis   :Overlay
      .RGB.I         :RGB   [x,y]   (R,G,B,A)
      .VectorField.I :VectorField   [x,y]   (Angle,Magnitude)
      .HLine.I       :HLine   [x,y]
      .VLine.I       :VLine   [x,y]
   .Overlay.Y_apostrophe_equals_x_sin_left_parenthesis_5_y_right_parenthesis :Overlay
      .RGB.I         :RGB   [x,y]   (R,G,B,A)
      .VectorField.I :VectorField   [x,y]   (Angle,Magnitude)
      .HLine.I       :HLine   [x,y]
      .VLine.I       :VLine   [x,y]

### Euler Integration Curves

In [7]:
def euler_step(x,y,h,func):
    '''x <- x +h, y_<- y + h f(x,y)'''
    hs = h * func(x,y)
    x = x + h
    y = y + hs
    return x,y

def euler_table(x0,y0,n,h,func):
    '''compute up to n euler steps with step size h for  y' = f(x,y) starting from (x0,y0)
    return the results in an hv.Table
    '''
    xl = [x0]; yl=[y0]
    for i in range(n):
        x0,y0 = euler_step(x0,y0,h,func)
        xl.append(x0);yl.append(y0)
        if np.abs(x0) > 5. or np.abs(y0) > 5. : break   # we ran off the grid
    return hv.Table(pd.DataFrame(dict(x=xl,y=yl)), kdims=['x'],vdims=['y'])

def euler_curve(x0,y0,n,h,func):
    '''compute up to n euler steps with step size h for  y' = f(x,y) starting from (x0,y0)
    return the results in an hv.Curve
    '''
    return euler_table(x0,y0,n,h,func).to.curve( label= 'h=%6.3f'%h)

def append_euler_plots( l, start, func, n=1000, h=[.5,.2,.01,.0011] ):
    for hi in h: l.append( euler_curve(*start, n, hi, func) )
    return l

In [8]:
# Test the computations and the resulting displays:
# --------------------------------------------------------------
f1 = lambda x,y: np.sin(x*y)
l1 = background(f1)
append_euler_plots(l1, (-5,np.pi/4.75),f1 )

f2 = lambda x,y: np.sin(x)*np.sin(y)
l2 = background(f2)
append_euler_plots(l2, (-5, np.pi/4.75),f2 )
append_euler_plots(l2, (-5,-np.pi/4.75),f2 )

#func = lambda x,y: np.sin(x*y)
#func = np.cos
#func = lambda x: np.exp(-x/( x**2 + 1))
#func = lambda x,y: x**2*np.sin(5*y) 
#func = lambda x,y: x*np.sin(5*y)
#func = lambda x,y: x*np.tan(y)
#func = lambda x,y: x/np.cosh(y)

# We need to call redim in case some curve overshot the grid
pos_opts = dict(legend_position='right', toolbar='above')
hv.Overlay(l1).redim.range(x=(-5,5),y=(-5,5)).opts(plot=pos_opts).relabel("y' = sin(x y)" ) +\
hv.Overlay(l2).redim.range(x=(-5,5),y=(-5,5)).opts(plot=pos_opts).relabel("y' = sin(x) sin(y)" )

:Layout
   .Overlay.Y_apostrophe_equals_sin_left_parenthesis_x_y_right_parenthesis                                        :Overlay
      .RGB.I                          :RGB   [x,y]   (R,G,B,A)
      .VectorField.I                  :VectorField   [x,y]   (Angle,Magnitude)
      .HLine.I                        :HLine   [x,y]
      .VLine.I                        :VLine   [x,y]
      .Curve.H_equals_0_full_stop_500 :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_200 :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_010 :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_001 :Curve   [x]   (y)
   .Overlay.Y_apostrophe_equals_sin_left_parenthesis_x_right_parenthesis_sin_left_parenthesis_y_right_parenthesis :Overlay
      .RGB.I                             :RGB   [x,y]   (R,G,B,A)
      .VectorField.I                     :VectorField   [x,y]   (Angle,Magnitude)
      .HLine.I                           :HLine   [x,y]
      .VLine.I                           :VLine   [x,y]
      .Curve.H_equals_0_full_stop_500.I  :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_200.I  :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_010.I  :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_001.I  :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_500.II :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_200.II :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_010.II :Curve   [x]   (y)
      .Curve.H_equals_0_full_stop_001.II :Curve   [x]   (y)

<span style="color:red;">Looks like the labels of the right are messed up</span>

## Add Streams

Want: click on a point in the x,y plane: compute and display Euler solutions for a set of step sizes h